# Multilayer perceptron

This software uses cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline 

See MIT License in 
https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline/README.md

## Data
Download the data from http://www.timeseriesclassification.com/
Save it in deepscent/data
E.g. deepscent/data/Adiac/Adiac_TRAIN.txt

In [17]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import utils
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pandas as pd

import os

np.random.seed(813306)

      
def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 5000

fdir = '../data'  
#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist = ['Adiac']
for each in flist:
    fname = each
    x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
    x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
    nb_classes =len(np.unique(y_test))
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    batch_size = min(x_train.shape[0]/10, 16)
    
    Y_train = utils.to_categorical(y_train, nb_classes)
    Y_test = utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
     
   # x_test_min = np.min(x_test, axis = 1, keepdims=1)
   # x_test_max = np.max(x_test, axis = 1, keepdims=1)
    x_test = (x_test - x_train_mean)/(x_train_std)
     
    #x_train = x_train.reshape(x_train.shape + (1,))
    #x_test = x_test.reshape(x_test.shape + (1,))
    
    x = Input(x_train.shape[1:])
    y= Dropout(0.1)(x)
    y = Dense(500, activation='relu')(x)
    y = Dropout(0.2)(y)
    y = Dense(500, activation='relu')(y)
    y = Dropout(0.2)(y)
    y = Dense(500, activation = 'relu')(y)
    y = Dropout(0.3)(y)
    out = Dense(nb_classes, activation='softmax')(y)
     
    model = Model(x, out)
     
    optimizer = keras.optimizers.Adadelta()    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=200, min_lr=0.1)
    
    hist = model.fit(x_train, Y_train, batch_size=batch_size, nb_epoch=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), 
                #callbacks = [TestCallback((x_train, Y_train)), reduce_lr, keras.callbacks.TensorBoard(log_dir='./log'+fname, histogram_freq=1)])
                 callbacks=[reduce_lr])
    
    #Print the testing results which has the lowest training loss.
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

 



Train on 390 samples, validate on 391 samples
Epoch 1/5000
390/390 [==============================] - 2s 5ms/step - loss: 3.6848 - acc: 0.0333 - val_loss: 3.5603 - val_acc: 0.0614
Epoch 2/5000
390/390 [==============================] - 0s 552us/step - loss: 3.4855 - acc: 0.0641 - val_loss: 3.4039 - val_acc: 0.0972
Epoch 3/5000
390/390 [==============================] - 0s 533us/step - loss: 3.3620 - acc: 0.0897 - val_loss: 3.3687 - val_acc: 0.0946
Epoch 4/5000
390/390 [==============================] - 0s 549us/step - loss: 3.3139 - acc: 0.1231 - val_loss: 3.5067 - val_acc: 0.0921
Epoch 5/5000
390/390 [==============================] - 0s 549us/step - loss: 3.2737 - acc: 0.1051 - val_loss: 3.3176 - val_acc: 0.1279
Epoch 6/5000
390/390 [==============================] - 0s 539us/step - loss: 3.1787 - acc: 0.1077 - val_loss: 3.1634 - val_acc: 0.1637
Epoch 7/5000
390/390 [==============================] - 0s 541us/step - loss: 3.1521 - acc: 0.1333 - val_loss: 3.3043 - val_acc: 0.1100
Epoc

Epoch 60/5000
390/390 [==============================] - 0s 546us/step - loss: 1.7078 - acc: 0.4256 - val_loss: 1.7671 - val_acc: 0.4373
Epoch 61/5000
390/390 [==============================] - 0s 550us/step - loss: 1.7179 - acc: 0.4179 - val_loss: 2.0963 - val_acc: 0.3402
Epoch 62/5000
390/390 [==============================] - 0s 541us/step - loss: 1.7385 - acc: 0.4256 - val_loss: 1.9550 - val_acc: 0.3862
Epoch 63/5000
390/390 [==============================] - 0s 546us/step - loss: 1.6155 - acc: 0.4436 - val_loss: 1.7470 - val_acc: 0.4169
Epoch 64/5000
390/390 [==============================] - 0s 547us/step - loss: 1.6586 - acc: 0.4231 - val_loss: 1.9337 - val_acc: 0.3913
Epoch 65/5000
390/390 [==============================] - 0s 542us/step - loss: 1.5992 - acc: 0.4897 - val_loss: 2.0281 - val_acc: 0.4092
Epoch 66/5000
390/390 [==============================] - 0s 541us/step - loss: 1.6718 - acc: 0.4231 - val_loss: 2.3950 - val_acc: 0.2558
Epoch 67/5000
390/390 [==================

390/390 [==============================] - 0s 549us/step - loss: 1.3058 - acc: 0.5333 - val_loss: 1.5247 - val_acc: 0.5320
Epoch 120/5000
390/390 [==============================] - 0s 549us/step - loss: 1.2375 - acc: 0.5359 - val_loss: 1.6139 - val_acc: 0.4783
Epoch 121/5000
390/390 [==============================] - 0s 544us/step - loss: 1.3530 - acc: 0.5128 - val_loss: 1.7786 - val_acc: 0.4629
Epoch 122/5000
390/390 [==============================] - 0s 546us/step - loss: 1.1379 - acc: 0.5795 - val_loss: 1.5618 - val_acc: 0.5192
Epoch 123/5000
390/390 [==============================] - 0s 542us/step - loss: 1.2613 - acc: 0.5564 - val_loss: 2.1879 - val_acc: 0.4041
Epoch 124/5000
390/390 [==============================] - 0s 537us/step - loss: 1.3679 - acc: 0.5333 - val_loss: 1.8617 - val_acc: 0.4808
Epoch 125/5000
390/390 [==============================] - 0s 545us/step - loss: 1.1720 - acc: 0.6051 - val_loss: 1.8386 - val_acc: 0.4629
Epoch 126/5000
390/390 [=========================

390/390 [==============================] - 0s 527us/step - loss: 1.0158 - acc: 0.6205 - val_loss: 1.8473 - val_acc: 0.5192
Epoch 179/5000
390/390 [==============================] - 0s 511us/step - loss: 1.1274 - acc: 0.5949 - val_loss: 1.7961 - val_acc: 0.4936
Epoch 180/5000
390/390 [==============================] - 0s 526us/step - loss: 1.0513 - acc: 0.6385 - val_loss: 1.6701 - val_acc: 0.5754
Epoch 181/5000
390/390 [==============================] - 0s 557us/step - loss: 1.0408 - acc: 0.6256 - val_loss: 1.9425 - val_acc: 0.4757
Epoch 182/5000
390/390 [==============================] - 0s 546us/step - loss: 1.1682 - acc: 0.6077 - val_loss: 2.2002 - val_acc: 0.4655
Epoch 183/5000
390/390 [==============================] - 0s 550us/step - loss: 0.9852 - acc: 0.6564 - val_loss: 2.3311 - val_acc: 0.4246
Epoch 184/5000
390/390 [==============================] - 0s 550us/step - loss: 0.9718 - acc: 0.6359 - val_loss: 1.8484 - val_acc: 0.5217
Epoch 185/5000
390/390 [=========================

390/390 [==============================] - 0s 549us/step - loss: 0.8667 - acc: 0.6872 - val_loss: 2.1409 - val_acc: 0.5141
Epoch 238/5000
390/390 [==============================] - 0s 542us/step - loss: 0.9130 - acc: 0.6744 - val_loss: 2.6287 - val_acc: 0.4220
Epoch 239/5000
390/390 [==============================] - 0s 545us/step - loss: 0.9487 - acc: 0.6513 - val_loss: 1.6481 - val_acc: 0.5754
Epoch 240/5000
390/390 [==============================] - 0s 553us/step - loss: 0.8839 - acc: 0.6769 - val_loss: 1.7879 - val_acc: 0.5934
Epoch 241/5000
390/390 [==============================] - 0s 540us/step - loss: 1.0549 - acc: 0.6359 - val_loss: 1.5339 - val_acc: 0.5831
Epoch 242/5000
390/390 [==============================] - 0s 539us/step - loss: 0.7947 - acc: 0.6949 - val_loss: 1.7394 - val_acc: 0.5601
Epoch 243/5000
390/390 [==============================] - 0s 542us/step - loss: 0.8792 - acc: 0.6795 - val_loss: 1.6364 - val_acc: 0.5985
Epoch 244/5000
390/390 [=========================

...

390/390 [==============================] - 0s 545us/step - loss: 0.0139 - acc: 0.9949 - val_loss: 2.3238 - val_acc: 0.6931
Epoch 4722/5000
390/390 [==============================] - 0s 543us/step - loss: 0.0078 - acc: 1.0000 - val_loss: 2.3620 - val_acc: 0.7008
Epoch 4723/5000
390/390 [==============================] - 0s 524us/step - loss: 0.0353 - acc: 0.9923 - val_loss: 2.3259 - val_acc: 0.6957
Epoch 4724/5000
390/390 [==============================] - 0s 544us/step - loss: 0.0356 - acc: 0.9923 - val_loss: 2.3378 - val_acc: 0.7059
Epoch 4725/5000
390/390 [==============================] - 0s 534us/step - loss: 0.0063 - acc: 1.0000 - val_loss: 2.3542 - val_acc: 0.6957
Epoch 4726/5000
390/390 [==============================] - 0s 537us/step - loss: 0.0159 - acc: 0.9949 - val_loss: 2.3946 - val_acc: 0.6982
Epoch 4727/5000
390/390 [==============================] - 0s 525us/step - loss: 0.0418 - acc: 0.9846 - val_loss: 2.3785 - val_acc: 0.7059
Epoch 4728/5000
390/390 [==================

390/390 [==============================] - 0s 532us/step - loss: 0.0162 - acc: 0.9923 - val_loss: 2.4282 - val_acc: 0.7008
Epoch 4781/5000
390/390 [==============================] - 0s 543us/step - loss: 0.0111 - acc: 1.0000 - val_loss: 2.4138 - val_acc: 0.6957
Epoch 4782/5000
390/390 [==============================] - 0s 540us/step - loss: 0.0274 - acc: 0.9949 - val_loss: 2.3979 - val_acc: 0.6957
Epoch 4783/5000
390/390 [==============================] - 0s 528us/step - loss: 0.0319 - acc: 0.9949 - val_loss: 2.3920 - val_acc: 0.6880
Epoch 4784/5000
390/390 [==============================] - 0s 528us/step - loss: 0.0181 - acc: 0.9974 - val_loss: 2.4049 - val_acc: 0.6931
Epoch 4785/5000
390/390 [==============================] - 0s 535us/step - loss: 0.0250 - acc: 0.9897 - val_loss: 2.3778 - val_acc: 0.6931
Epoch 4786/5000
390/390 [==============================] - 0s 547us/step - loss: 0.0211 - acc: 0.9949 - val_loss: 2.3991 - val_acc: 0.6931
Epoch 4787/5000
390/390 [==================

390/390 [==============================] - 0s 541us/step - loss: 0.0143 - acc: 0.9949 - val_loss: 2.4027 - val_acc: 0.6957
Epoch 4840/5000
390/390 [==============================] - 0s 536us/step - loss: 0.0061 - acc: 1.0000 - val_loss: 2.4222 - val_acc: 0.6957
Epoch 4841/5000
390/390 [==============================] - 0s 542us/step - loss: 0.0098 - acc: 1.0000 - val_loss: 2.4137 - val_acc: 0.7059
Epoch 4842/5000
390/390 [==============================] - 0s 548us/step - loss: 0.0394 - acc: 0.9821 - val_loss: 2.4488 - val_acc: 0.6982
Epoch 4843/5000
390/390 [==============================] - 0s 540us/step - loss: 0.0345 - acc: 0.9949 - val_loss: 2.4707 - val_acc: 0.7008
Epoch 4844/5000
390/390 [==============================] - 0s 543us/step - loss: 0.0332 - acc: 0.9872 - val_loss: 2.4297 - val_acc: 0.7008
Epoch 4845/5000
390/390 [==============================] - 0s 543us/step - loss: 0.0332 - acc: 0.9923 - val_loss: 2.4250 - val_acc: 0.6905
Epoch 4846/5000
390/390 [==================

390/390 [==============================] - 0s 534us/step - loss: 0.0083 - acc: 0.9974 - val_loss: 2.3694 - val_acc: 0.6982
Epoch 4899/5000
390/390 [==============================] - 0s 551us/step - loss: 0.0119 - acc: 0.9949 - val_loss: 2.4206 - val_acc: 0.6982
Epoch 4900/5000
390/390 [==============================] - 0s 543us/step - loss: 0.0257 - acc: 0.9949 - val_loss: 2.3875 - val_acc: 0.7084
Epoch 4901/5000
390/390 [==============================] - 0s 546us/step - loss: 0.0173 - acc: 0.9949 - val_loss: 2.4142 - val_acc: 0.7084
Epoch 4902/5000
390/390 [==============================] - 0s 542us/step - loss: 0.0234 - acc: 0.9923 - val_loss: 2.3968 - val_acc: 0.6982
Epoch 4903/5000
390/390 [==============================] - 0s 531us/step - loss: 0.0263 - acc: 0.9897 - val_loss: 2.4100 - val_acc: 0.7059
Epoch 4904/5000
390/390 [==============================] - 0s 539us/step - loss: 0.0341 - acc: 0.9897 - val_loss: 2.4049 - val_acc: 0.6982
Epoch 4905/5000
390/390 [==================

390/390 [==============================] - 0s 549us/step - loss: 0.0203 - acc: 0.9923 - val_loss: 2.4217 - val_acc: 0.6905
Epoch 4958/5000
390/390 [==============================] - 0s 527us/step - loss: 0.0552 - acc: 0.9974 - val_loss: 2.4379 - val_acc: 0.6982
Epoch 4959/5000
390/390 [==============================] - 0s 544us/step - loss: 0.0223 - acc: 0.9949 - val_loss: 2.4548 - val_acc: 0.7059
Epoch 4960/5000
390/390 [==============================] - 0s 532us/step - loss: 0.0444 - acc: 0.9821 - val_loss: 2.4966 - val_acc: 0.7033
Epoch 4961/5000
390/390 [==============================] - 0s 534us/step - loss: 0.0305 - acc: 0.9897 - val_loss: 2.4469 - val_acc: 0.6982
Epoch 4962/5000
390/390 [==============================] - 0s 543us/step - loss: 0.0305 - acc: 0.9923 - val_loss: 2.4689 - val_acc: 0.7059
Epoch 4963/5000
390/390 [==============================] - 0s 536us/step - loss: 0.0216 - acc: 0.9923 - val_loss: 2.4284 - val_acc: 0.7033
Epoch 4964/5000
390/390 [==================